In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from IPython.display import HTML # intégration notebook
%matplotlib inline

def show_colors(colors):
    """
    Draw a square for each color contained in the colors list
    given in argument.
    """
    with plt.rc_context(plt.rcParamsDefault):
        fig = plt.figure(figsize=(6, 1), frameon=False)
        ax = fig.add_subplot(111)
        for x, color in enumerate(colors):
            ax.add_patch(
                mpl.patches.Rectangle(
                    (x, 0), 1, 1, facecolor=color
                )
            )
        ax.set_xlim((0, len(colors)))
        ax.set_ylim((0, 1))
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_aspect("equal")
    
    return fig

In [2]:
symbology = [
    [[255, 171, 187, 100], 999999, '> 20"'],
    [[255, 254, 3, 100], 20, '15 - 20"'],
    [[255, 213, 0, 100], 15, '10 - 15"'],
    [[204, 133, 0, 100], 10, '7 - 10"'],
    [[255, 124, 10, 100], 7, '5 - 7"'],
    [[240, 62, 4, 100], 5, '4 - 5"'],
    [[205, 0, 0, 100], 4, '3 - 4"'],
    [[139, 0, 0, 100], 3, '2.5 - 3"'],
    [[125, 0, 125, 100], 2.5, '2 - 2.5"'],
    [[125, 0, 125, 100], 2, '1.75 - 2"'],
    [[124, 94, 185, 100], 1.75, '1.5 - 1.75"'],
    [[0, 238, 238, 100], 1.5, '1.25 - 1.5"'],
    [[0, 161, 215, 100], 1.25, '1 - 1.25"'],
    [[27, 130, 230, 100], 1, '0.75 - 1"'],
    [[14, 70, 125, 100], .75, '0.5 - 0.75"'],
    [[0, 139, 0, 100], .5, '0.25 - 0.5"'],
    [[0, 185, 0, 100], .25, '0.1 - 0.25"'],
    [[127, 255, 0, 100], .1, '0.01 - 0.1"']
]
symbology.reverse()

rgb_scale = [class_break[0] for class_break in symbology]
color_scale = ['#%02x%02x%02x' % tuple(class_break[:3]) for class_break in rgb_scale]
bound_scale = [class_break[1] for class_break in symbology]
labels = [class_break[2] for class_break in symbology]
print(bound_scale)
print(labels)
show_colors(color_scale)

[0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 1.75, 2, 2.5, 3, 4, 5, 7, 10, 15, 20, 999999]
['0.01 - 0.1"', '0.1 - 0.25"', '0.25 - 0.5"', '0.5 - 0.75"', '0.75 - 1"', '1 - 1.25"', '1.25 - 1.5"', '1.5 - 1.75"', '1.75 - 2"', '2 - 2.5"', '2.5 - 3"', '3 - 4"', '4 - 5"', '5 - 7"', '7 - 10"', '10 - 15"', '15 - 20"', '> 20"']


<Figure size 600x100 with 1 Axes>

In [4]:
import arcpy
aprx = arcpy.mp.ArcGISProject("CURRENT")

pro_map = aprx.listMaps()[0]
class_breaks = [{
    "type" : "CIMRasterClassBreak",
    "upperBound" : value,
    "label" : label,
    "color" : {"type" : "CIMRGBColor", "values" : color}
} for color, value, label in symbology]

colorizer = {
        "type" : "CIMRasterClassifyColorizer",
        "resamplingType" : "NearestNeighbor",
        "normalizationType" : "Nothing",
        "classificationMethod" : "Manual",
        "classBreaks" : class_breaks,
        "colorRamp" : {
          "type" : "CIMLinearContinuousColorRamp",
          "colorSpace" : {
            "type" : "CIMICCColorSpace",
            "url" : "Default RGB"
          },
          "fromColor" : {
            "type" : "CIMHSVColor",
            "values" : [
              60,
              100,
              96,
              100
            ]
          },
          "toColor" : {
            "type" : "CIMHSVColor",
            "values" : [
              0,
              100,
              96,
              100
            ]
          }
        },
}

print('Updating the connectionProperties of each layer...')
for layer in pro_map.listLayers():
    if not layer.connectionProperties:
        continue
    print(layer.name)
    layerCIM = layer.getDefinition('V2')
    
    layerCIM.colorizer = colorizer

    layer.setDefinition(layerCIM)
    
    if layer.isRasterLayer:
        continue

aprx.save()

Updating the connectionProperties of each layer...
past_1hour_accum_precipitation.mrf
past_3hour_accum_precipitation.mrf
past_6hour_accum_precipitation.mrf
past_12hour_accum_precipitation.mrf
past_24hour_accum_precipitation.mrf
past_48hour_accum_precipitation.mrf
past_72hour_accum_precipitation.mrf
